In [2]:
!pip install wbdata duckdb

import wbdata
import duckdb
import pandas as pd
import datetime
countries = ['AF', 'GB', 'IN', 'KE', 'BR']

indicators = {
    'SP.DYN.LE00.IN': 'life_expectancy',
    'SH.DYN.MORT': 'under5_mortality',
    'SP.ADO.TFRT': 'adolescent_fertility',
    'SE.SEC.ENRR': 'secondary_school_enrollment'
}

df = wbdata.get_dataframe(indicators, country=countries)
df = df.reset_index()

df['year'] = pd.to_datetime(df['date']).dt.year

df = df[(df['year'] >= 2000) & (df['year'] <= 2023)]

In [3]:

duckdb.sql("CREATE OR REPLACE VIEW wdi_raw AS SELECT * FROM df")

df_clean = duckdb.sql("""
SELECT
  country,
  year,
  MAX(life_expectancy) AS life_expectancy,
  MAX(under5_mortality) AS under5_mortality,
  MAX(adolescent_fertility) AS adolescent_fertility,
  MAX(secondary_school_enrollment) AS secondary_school_enrollment
FROM wdi_raw
GROUP BY country, year
ORDER BY country, year
""").df()

df_clean

,country,year,life_expectancy,under5_mortality,adolescent_fertility,secondary_school_enrollment
0,Afghanistan,2000,55.005000,131.7,154.310,NaN
1,Afghanistan,2001,55.511000,127.4,152.471,14.471510
2,Afghanistan,2002,56.225000,123.1,150.161,NaN
3,Afghanistan,2003,57.171000,118.7,143.750,14.078050
4,Afghanistan,2004,57.810000,114.2,136.930,19.263140
...,...,...,...,...,...,...
115,United Kingdom,2019,81.369122,4.4,11.064,117.233337
116,United Kingdom,2020,80.331756,4.4,9.701,114.651649
117,United Kingdom,2021,80.647317,4.4,8.578,113.683823
118,United Kingdom,2022,81.011220,4.5,8.379,113.228363


In [5]:
df_clean.to_csv("../data/population_dynamics_clean.csv", index=False)